In [14]:
#********************************************************************
# Note: all calculations done in SI units
# COOL 2D array stuff
#********************************************************************

#Global Constants
kB = 1.38064852*10**(-23) # m^2 kg s^-2 K^-1 (boltzmann constant)
M  = 171*1.66054e-27      # mass of Yb in kg

#Libraries
import matplotlib.pyplot as plt #plotting
import numpy as np
import matplotlib.pylab as pylab #plotting
import scipy.integrate as integrate #integrating
from numpy import sqrt, sin, cos, pi
from decimal import Decimal

#Set Parameters
T_oven       = 400        # Temperature of the Oven in celcius
len_algn_fud = 3          # length of the triangle alignment fudge factor

#Dimensional Parameters (m)
R_DPT      = 2.5*10**(-3) # radius of DPT
L_DPT      = 518*10**(-3) # length from the end of nozzle to the end of DPT   
trngle_len = 2*np.sqrt(3)*R_DPT

# outer diameter radius and inner array parameters picked from the sheet
rOD = np.array([0.0425,0.0355,0.032,0.028,0.025,0.020,0.0163,0.008])*(0.0254/2) #conversion to meters and radius
rID = np.array([0.035 ,0.031 ,0.029,0.024,0.020,0.016,0.0120,0.006])*(0.0254/2) #conversion to meters amd radius

# Convert set temperature in Celcius to Kelvin
T = T_oven + 273

# Pressure, velocities, densities
P        = 10**(5.006 + 9.111 - 8111/T -1.0849*np.log10(T)) #pressure calculation
v_tilde  = np.sqrt(kB*T/M)                                  # convenient redefinition
n_0      = P/(kB*T)                                         # Number of atoms per Volume
v_mp     = np.sqrt(2)*v_tilde                               # most probable velocity
v_avg    = np.sqrt(8/(np.pi))*v_tilde                       # avergae velocity
v_rms    = np.sqrt(3)*v_tilde                               # root-mean-square velocity

# Calculating Dimensional Parameters
N_tubes      = (trngle_len/(2*rOD))*((trngle_len/(2*rOD))+1) # number of tubes
A_trngle     = (trngle_len**(2)*sqrt(3)/4)                 # area of triangle nozzle without tubes
A_open       = A_trngle - N_tubes*pi*(rOD**2 - rID**2)     # m^2, total open area of nozzle
L_tubes      = np.linspace(1, 40, 100)*10**(-3)            # range of possibly length of tube values
theta_DPT    = np.arctan(R_DPT/L_DPT)                      # optimal length of tubing

#Find the element of L_tubes array which makes theta_mx and theta_DPT as close as possible
chk_array    = np.array([]) 
for i in range (0,len(rID)):    
    for j in range (0,len(L_tubes)):
        theta_mx     = np.arctan(rID[i]/L_tubes[j])
        chk_angles   = np.abs(theta_mx - theta_DPT) # subtraction array
        chk_array    = np.append(chk_array, chk_angles)

# minimum value obtained from the subtraction
min_sub_result = np.min(chk_array) # if theta_DPT = theta_mx, the subtraction of the two should be close to zero

# change back into a 2D array
nw_chk = np.reshape(chk_array,(-1,len(L_tubes)))

# Use indices to find optimal radius and length
i = np.where(nw_chk == min_sub_result)[0][0] #radius
j = np.where(nw_chk == min_sub_result)[1][0]

# calculate the correct theta max
theta_mx     = np.arctan(rID[i]/L_tubes[j])
theta_mx_fud = np.arctan(rID[i]/(L_tubes[j]/len_algn_fud))

# Area Ratio of theta_DPT and theta_mx
r_sld_angles     = (theta_DPT)**2/(theta_mx)**2 
r_sld_angles_fud = (theta_DPT)**2/(theta_mx_fud)**2

print('inner diameter in inches')
print(rID[i]*2/0.0254)
print('      ')

print('outer diameter in inches')
print(rOD[i]*2/0.0254)
print('      ')

print('tube length in mm')
print(L_tubes[j]*10**3)
print('      ')

print('Number of tubes necessary')
print(N_tubes[i])
print('      ')

print('Length of the triangle in mm')
print(trngle_len*10**(3))
print('      ')

print('Length of the tubes (fudged) in mm')
print(L_tubes[j]*10**(3)/len_algn_fud)
print('      ')

# Flux Calculations
I        = integrate.quad(lambda x: sin(x)*cos(x)/2, 0, theta_mx) 
Flux     = A_open[i]*v_mp*n_0*I[0]                       
I_fud    = integrate.quad(lambda x: sin(x)*cos(x)/2, 0, theta_mx_fud) 
Flux_fud = A_open[i]*v_mp*n_0*I_fud[0] 
Flux_DPT     = Flux*r_sld_angles
Flux_DPT_fud = Flux*r_sld_angles_fud

print('Flux: without the fudge (after nozzle)')
print('%.2E' % Decimal(Flux))
print('          ')
print('Flux: with the fudge (after nozzle)')
print('%.2E' % Decimal(Flux_fud))
print('          ')

# Flux after the DPT
print('Flux: without the fudge (out of DPT)')
print('%.2E' % Decimal(Flux_DPT))
print('          ')
print('Flux: with the fudge (out of DPT)')
print('%.2E' % Decimal(Flux_DPT_fud))
#**********************************************************
#    Possible updates:
# write what the parameters correspond to in terms of gauge
# modify the distribution of atoms
#**********************************************************

inner diameter in inches
0.012
      
outer diameter in inches
0.0163
      
tube length in mm
31.72727272727273
      
Number of tubes necessary
458.45834280384713
      
Length of the triangle in mm
8.660254037844387
      
Length of the tubes (fudged) in mm
10.575757575757576
      
Flux: without the fudge (after nozzle)
6.63E+10
          
Flux: with the fudge (after nozzle)
5.97E+11
          
Flux: without the fudge (out of DPT)
6.70E+10
          
Flux: with the fudge (out of DPT)
7.44E+09
